In [1]:
from DATASET import SOURCE_A, SOURCE_B, SOURCE_C

In [2]:
VEGAN_NON_VEGAN_INGREDIENTS = SOURCE_A + SOURCE_B + SOURCE_C

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer

_EMB = SentenceTransformer("all-mpnet-base-v2")


def _e(txt: str) -> np.ndarray:
    return _EMB.encode([txt], normalize_embeddings=True)[0]

In [4]:
from tqdm import tqdm

v_embs, nv_embs = [], []
v_txts, nv_txts = [], []
for txt, lbl in tqdm(VEGAN_NON_VEGAN_INGREDIENTS):
    prompt = "VEGAN PRODUCT: " if lbl else "ANIMAL PRODUCT: "
    vec = _e(prompt + txt)
    (v_embs if lbl else nv_embs).append(vec)
    (v_txts if lbl else nv_txts).append(txt)

100%|██████████| 5294/5294 [05:06<00:00, 17.25it/s]


In [5]:
EMB_ALL = np.vstack([v_embs, nv_embs])
ALL_INGREDIENTS = np.concat([v_txts, nv_txts])

In [6]:
np.save("vegan_dependencies/vegan_embeddings.npy", EMB_ALL)
np.save("vegan_dependencies/vegan_ingredients.npy", np.array(v_txts))
np.save("vegan_dependencies/non_vegan_ingredients.npy", np.array(nv_txts))

In [7]:
temp = _e("cilantro")

scores = EMB_ALL @ temp

best_idx   = int(np.argmax(scores))
best_score = float(scores[best_idx])
closest = ALL_INGREDIENTS[best_idx]
print(f"closest ingredient is: {closest} | score: {best_score:.3f}")

closest ingredient is: fresh cilantro | score: 0.634
